<a href="https://colab.research.google.com/github/sriram-lab/utilities/blob/master/Metabolomics_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Metabolomics parser

`metabolomics-parser.py` uses the common metabolite names in a metabolomics dataset and maps them to standard
metabolite identifiers in established databases. This is used for mapping metabolites to constraint-based metabolic
models (sbml-based namespaces only).

**Author:** Scott Campit

In [13]:
import sys, os
import re
import pandas as pd
import numpy as np

# Install dependencies
!pip3 install pubchempy
!pip3 install cobra
!pip3 install lxml
!pip3 install xlsxwriter

  Created wheel for pubchempy: filename=PubChemPy-1.0.4-cp36-none-any.whl size=13825 sha256=942d323a68de963154e99e631f42280265f90b077a14d9a9a1d57fd24ed9ed94
  Stored in directory: /root/.cache/pip/wheels/10/4d/51/6b843681a9a5aef35f0d0fbce243de46f85080036e16118752
Successfully built pubchempy
     |████████████████████████████████| 2.4MB 8.2MB/s 
     |████████████████████████████████| 112kB 48.2MB/s 
     |████████████████████████████████| 133kB 53.9MB/s 
     |████████████████████████████████| 634kB 49.7MB/s 
     |████████████████████████████████| 44.1MB 69kB/s 
     |████████████████████████████████| 552kB 58.6MB/s 
     |████████████████████████████████| 143kB 9.4MB/s 


## Query the PubChem database for metabolite name synoynms that match with the metabolomics

`queryPubChem` maps the metabolite name from a Pandas dataframe in the `Compound Method` column and extracts synoynms from several databases using the PubChem API.

**INPUTS:**
  * `data:` A Pandas dataframe of the metabolomics dataframe with common metabolite identifiers.
    * **NOTE:** Common metabolite names need to be under the `Compound Method` column name in the Pandas dataframe. 
  * `filename:` A string denoting the path to the metabolomics dataset.
  * `sheet:` A string denoting the sheet name from the metabolomics file. By default, the sheet name that is read in is `Sheet1`.

**OUTPUTS:**
  * `all_compoounds:` A Pandas series containing the all metabolite synoynms as a single column. 
  * `queryList:` A list containing all metabolite synoynms.

In [2]:
def queryPubChem(data, filename='', sheet='Sheet1'):
    """
    queryPubChem maps the metabolite name from a pandas Dataframe in the 'Compound Method' column and extracts
    synoynms from several databases using the PubChem API.
    :param data:           A Pandas Dataframe of the metabolomics dataframe with the common metabolite identifiers
                           under the 'Compound Method' column
    :return all_compounds: A Pandas Dataframe from the PubChem API containing the metabolite map. This dataframe is
                           saved as a .csv file.
    :return queryList:     A string with semicolon delimters to be fed into a REST-API
    """
    import pubchempy as pcp
    import pandas.api.types as ptypes

    # Split 'Compound Method' column by the '/' regex and clean up some data
    data = data.drop('Compound Method', axis=1).join(data['Compound Method']
                                                 .str.split('/', expand=True)
                                                 .stack().reset_index(level=1, drop=True)
                                                 .rename('Metabolite'))
    data['Metabolite'] = data['Metabolite'].str.lower()
    pubChemQuery = data['Metabolite'].tolist()

    # Mine the PubChem database for synonyms
    all_compounds = []
    print("Mapping metabolite names to PubChem database for synonym matching and ID retrieval.")
    for metabolite in pubChemQuery:
        try:
            df = pcp.get_substances(identifier=metabolite,
                                    namespace='name',
                                    as_dataframe=True)
            df['Name'] = metabolite
            df = df.applymap(str)
            print(df)
            all_compounds.append(df)
        except (KeyError, TimeoutError, pcp.TimeoutError):
            continue
    all_compounds = pd.concat(all_compounds)
    #all_compounds.to_csv(filename + '_' + sheet +'_pubchem.csv')

    # Read in assembled database as pandas dataframe. It transforms the nested list in the Pandas Dataframe to a
    # string that can be manipulated using regexs
    #all_compounds = pd.read_csv(filename + '_' + sheet +'_pubchem.csv')

    # Explode synonyms column
    all_compounds = all_compounds.drop('synonyms', axis=1).join(all_compounds['synonyms']
                                                              .str.split(',', expand=True)
                                                              .stack().reset_index(level=1, drop=True)
                                                              .rename('synonyms'))

    # Clean up regexes and get unique compounds
    patterns = ["[", "]", "'"]
    for p in patterns:
        all_compounds['synonyms'] = all_compounds['synonyms'].str.replace(p, '')
    all_compounds['synonyms'] = all_compounds['synonyms'].str.lower()

    # Make this JSON serializable
    queryList = ';'.join(all_compounds['synonyms'].unique())
    return all_compounds, queryList

## Query MetaboAnalyst database to get matching systematic identifiers using all synonyms

`queryMetaboAnalyst` takes in the filepath corresponding to the metabolomics data and uses the first column as metabolite names in the file. 

**NOTE:** This function uses the `queryPubChem` function, and needs the first column in the metabolomics data file to be named `Compound Method`.

**INPUTS:** 
  * `filename:` A string denoting the path to the metabolomics dataset.
  * `sheet:` A string denoting the sheet name from the metabolomics file. By default, the sheet name that is read in is `Sheet1`.

**OUTPUT:**
  * `data:` A Pandas dataframe containing all systematic metabolite IDs

In [4]:
def queryMetaboAnalyst(filename='', sheet='Sheet1'):
    """
    queryMetaboAnalyst takes the file (currently only written for csv files),
    and uses the first column as metabolite names in the file. 
    
    The metabolite names are queried using MetaboAnalyst. The output is a metabolite map consisting of
    several different identifiers for the given metabolite.

    :param   filename: A string denoting the path to a text delimited or Excel file containing the metabolomics data
    :return: data:     A Pandas dataframe with a metabolite map containing several different metabolite identifiers
    """

    import requests
    import json

    print('Mapping metabolomics data to additional identifiers')
    #if os.path.splitext(filename)[1] is '.xls' or 'xlsx':
    #    fileData = pd.read_excel(filename, sheet_name=sheet)
    #else:
    #    fileData = pd.read_csv(filename)
    wb = gc.open_by_url(filename)
    wks = wb.worksheet(sheet)

    data = wks.get_all_values()
    fileData = pd.DataFrame(data)
    header = fileData.iloc[0] 
    fileData = fileData[1:] 
    fileData.columns = header

    # Get synonyms of each metabolite from PubChem
    all_compounds, queryList = queryPubChem(fileData, filename, sheet)

    # Query metaboAnalyst for additional database identifiers
    queryDict = {"queryList" : queryList,
             "inputType" : "name"}
    metabolites = json.dumps(queryDict)

    # Query MetaboAnalyst using data set metabolites
    url = "http://api.xialab.ca/mapcompounds"
    headers = {
        'Content-Type': "application/json",
        'cache-control': "no-cache",
    }
    response = requests.request("POST", url, data=metabolites, headers=headers).json()
    data = pd.DataFrame(response)

    identifiers = ['hmdb_id', 'kegg_id', 'pubchem_id', 'chebi_id', 'chebi_id', 'metlin_id']
    for id in identifiers:
        data[id] = data[id].replace('-', np.nan)
    data['chebi_id'] = data['chebi_id'].astype(float)

    print('MetaboAnalyst query done!')
    print(data)

    return data



## Extracting metabolite identifiers from RECON1


`mapMetabolicModel` takes in a metabolic model (xml format only), and parses the ChEBI, HMDBidentifiers in the map. 
    
**NOTE:** Currently only tested with RECON1 - xml namespaces will change depending on sbml specifications per metabolic models.

**INPUT:**
  * `model:` A string describing the path to the COBRA metabolic model file (`.xml` or `.sbml` file types supported only.

**OUTPUT:**
  * `modelMap:` A Pandas dataframe containing the metabolite identifiers for each metabolite in the metabolic model.


In [5]:
def mapMetabolicModel(model):
    """
    mapMetabolicModel takes in a metabolic model (xml format only), and parses the
    identifiers in the map. 
    
    Currently only tested with RECON1 - xml namespaces will change depending on metabolic models.

    :param  model:    A string describing the path to the metabolic model file (`.xml` or `.sbml` file types supported only.
    :return modelMap: A Pandas dataframe containing metabolite identifiers from the metabolic model
    """

    from lxml import etree
    print('Parsing metabolic model to get metabolite and associated identifiers')
    context = etree.parse(model)

    # Namespaces for different databases in the sbml models
    chebi_pattern = re.compile(r'.*http://identifiers.org/chebi/CHEBI:(\d+)')
    hmdb_pattern = re.compile(r'.*http://identifiers.org/hmdb/HMDB(\d+)')
    kegg_pattern = re.compile(r'.*http://identifiers.org/kegg.compound/C(\d+)')

    modelMap = pd.DataFrame()
    for metabolite in context.iter(tag='{http://www.sbml.org/sbml/level3/version1/core}species'):
        name = metabolite.get("name")
        bigg = metabolite.get("metaid")
        for element in metabolite.iter():
            if element.tag == '{http://www.w3.org/1999/02/22-rdf-syntax-ns#}li':
                content = etree.tostring(element)
                content_string = content.decode("utf-8")

                metabolite_name = []
                chebi = []
                hmdb = []
                kegg = []

                metabolite_name.append(name)
                metabolite_name = str(metabolite_name).replace(
                    '[', '').replace(']', '')

                if re.match(chebi_pattern, content_string):
                    chebi_num = re.findall(chebi_pattern, content_string)
                    chebi.append(chebi_num)
                else:
                    chebi.append(np.nan)
                chebi = str(chebi).replace('[', '').replace(']', '')

                if re.match(hmdb_pattern, content_string):
                    hmdb_num = re.findall(hmdb_pattern, content_string)
                    hmdb_num = ["HMDB" + h for h in hmdb_num]
                    hmdb.append(hmdb_num)
                else:
                    hmdb.append(np.nan)
                hmdb = str(hmdb).replace('[', '').replace(']', '')

                if re.match(kegg_pattern, content_string):
                    kegg_num = re.findall(kegg_pattern, content_string)
                    kegg_num = ["C" + k for k in kegg_num]
                    kegg.append(kegg_num)
                else:
                    kegg.append(np.nan)
                kegg = str(kegg).replace('[', '').replace(']', '')

                # Format stuff correctly before saving
                modelMap = modelMap.append({'Metabolite':metabolite_name,
                                            'BIGG':bigg, 'HMDB':hmdb,
                                            'CHEBI':chebi, 'KEGG':kegg},
                                            ignore_index=True)
        element.clear()

    modelMap['CHEBI'] = modelMap['CHEBI'].str.replace("'", "")
    modelMap['CHEBI'] = modelMap['CHEBI'].astype(float)
    print("Metabolic map complete")
    return modelMap

## Match model identifiers with the dataset

`matchModelAndData` uses the identifiers from the metabolomics data and the model to find matches between them.

**INPUTS:**
  * `data:` A Pandas dataframe containing the matched metabolite identifiers queried from MetaboAnalyst.
  * `modelMap:` A Pandas dataframe containing the mapped metabolite names from the COBRA metabolic model.

**OUTPUT:**
  * `mergedModelDataMap:` A Pandas dataframe of the merged map between the metabolomics data and the COBRA metabolic model.


In [6]:
def matchModelAndData(data, modelMap):
    """
    matchModelAndData uses the identifiers from the metabolomics data and the model
    to find matches between them.

    :param  data:               A Pandas dataframe containing data queried from MetaboAnalyst.
    :param  modelMap:           A Pandas dataframe queried from mapping the metabolite names to the COBRA metabolic
                                model.
    :return mergedModelDataMap: A Pandas dataframe of the merged map between the metabolomics data and the
                                metabolic model.
    """

    print('Match metabolomics identifiers and model identifiers by ChEBI and KEGG IDs')
    chebi = pd.merge(modelMap, data, left_on='CHEBI', right_on='chebi_id')
    chebi = chebi[np.isfinite(chebi['CHEBI'])]
    chebi = chebi[["Metabolite", "query", "BIGG", "CHEBI"]]

    kegg = pd.merge(modelMap, data, left_on='KEGG', right_on='kegg_id')
    kegg = kegg[pd.notnull(kegg['KEGG'])]
    kegg = kegg[['Metabolite', 'query', 'BIGG', 'KEGG']]

    mergedModelDataMap = pd.merge(chebi, kegg,
                                  how='outer', on=['Metabolite', 'query', 'BIGG'])
    mergedModelDataMap = mergedModelDataMap.drop_duplicates(keep='first')

    print("Found matching metabolites based on ChEBI and KEGG identities!")
    return mergedModelDataMap

## Map metabolite positions from metabolic model to the dataset

`mapMetabolitePositionsInModel` gets the metabolite positions using the `cobrapy` library.

**INPUTS:**
  * `mergedModelDataMap:` A Pandas dataframe of the merged map between the metabolomics data and the COBRA metabolic model.
  * `model:` A string denoting the path to the COBRA metabolic model ()

**OUTPUT:**
  * `PositionModel:` A Pandas dataframe containing the positions for each metabolite in the metabolic model.

In [7]:
def mapMetabolitePositionsInModel(mergedModelDataMap, model):
    """
    mapMetabolitePositionsInModel gets the metabolite positions using the cobrapy library.

    :param  mergedModelDataMap: A Pandas dataframe of the merged map between the metabolomics data and the metabolic
                                model.
    :param  model:              A string denoting the path to the metabolic model (`.xml` or `.sbml` file types supported only.
    :return PositionModel:      A Pandas dataframe containing the positions for each metabolite in the metabolic model.
    """

    import cobra
    print("Mapping metabolite positions in metabolic model")
    bigg_ids = mergedModelDataMap['BIGG'].replace('M_', '')
    bigg_ids = list(bigg_ids)
    biggRxn = pd.DataFrame()
    mdl = cobra.io.read_sbml_model(model)

    for index, met in enumerate(mdl.metabolites):
        if any(str(met) in m for m in bigg_ids):
            biggRxn = biggRxn.append(pd.DataFrame({"Position": index,
                                                   "Metabolite": [met]}
                                                  ), ignore_index=True)

    biggRxn['Metabolite'] = biggRxn['Metabolite'].astype(str)
    biggRxn["Compartment"] = biggRxn["Metabolite"].str.rsplit('_').str[-1]
    biggRxn["Metabolite"] = biggRxn["Metabolite"].str.split('_').str[0]

    biggRxn = biggRxn.pivot_table(index='Metabolite',
                            columns='Compartment',
                            values='Position',
                            aggfunc='mean', fill_value=0)

    mergedModelDataMap['BIGG'] = mergedModelDataMap['BIGG'].str.split('_', n=1).str[-1]
    mergedModelDataMap['BIGG'] = mergedModelDataMap['BIGG'].str.rsplit('_', n=1).str[0]
    mergedModelDataMap = mergedModelDataMap.drop(['CHEBI', 'KEGG', 'Metabolite'], axis=1)
    mergedModelDataMap = mergedModelDataMap.drop_duplicates(keep='first')

    PositionModel = pd.merge(biggRxn, mergedModelDataMap,
                              left_index=True, right_on='BIGG')
    PositionModel = PositionModel.drop(['BIGG'], axis=1)
    PositionModel = PositionModel.set_index(['query'])

    print('Mapped metabolite positions in metabolic model to metabolite name')
    return PositionModel

## Construct the final dataset used for Dynamic Flux Analysis

`constructFinalDataset` merges the array of metabolite positions and the file name together. It automatically outputs an Excel file, ready for piping into DFA.

**INPUTS:**
  * `PositionModel:` A Pandas dataframe containing the metabolite positions in the metabolic model. 
  * `filename:` A string denoting the path of the metabolomics data.
  * `sheet:` A string denoting the sheet name from the Excel sheet, if the file specified is an Excel file.

**OUTPUT:**
  * An Excel sheet containing the mapped data.

In [8]:
def constructFinalDataset(PositionModel, filename, sheet='Sheet1'):
    """
    constructFinalDataset merges the array of metabolite positions and the file name together.
    It automatically outputs an Excel file, ready for piping into DFA.
    
    :param  PositionModel: A Pandas dataframe containing the metabolite positions in the metabolic model.
    :param  filename:      A string denoting the path of the metabolomics data.
    :return df:            A Pandas dataframe containing the metabolomics data that intersects with the metabolic
                           model and the metabolite positions in the metabolomic model.
    """
    
    print("Merging metabolomics data to model map")
    #if os.path.splitext(filename)[1] is '.xls' or 'xlsx':
    #    fileData = pd.read_excel(filename, sheet_name=sheet)
    #else:
    #    fileData = pd.read_csv(filename)
    wb = gc.open_by_url(filename)
    wks = wb.worksheet(sheet)

    data = wks.get_all_values()
    fileData = pd.DataFrame(data)
    header = fileData.iloc[0] 
    fileData = fileData[1:] 
    fileData.columns = header

    df = pd.merge(PositionModel, fileData,
                  left_index=True, right_on=fileData.iloc[:, 0],
                  how='inner')    
    return df

Now that we have a function that uses all of the accessory functions written above, let's get our final metabolomics dataset from the ME1 dataset. First we need to mount our Google Drive account.

In [9]:
# This bit of code adds your Google Drive files into /content
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now let's install some stuff that let's us read in Google sheets.

In [10]:
# Install gspread if you haven't already
#!pip install --upgrade -q gspread
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

Next, let's call our files into the function.


In [11]:
model = r'/content/drive/My Drive/work/data/MetabolicModels/RECON1/RECON1.xml'
name = r'https://docs.google.com/spreadsheets/d/15Ma7bbSP1B6j6U9SDN6lrxwR3jIv6o6xRAYfnKGCUhk/edit?usp=sharing'

Finally, we can run our parser.

In [ ]:
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
from googleapiclient import discovery

wb = gc.open_by_url(name)
tmp = wb.worksheets()
tmp = tmp[1:]

sheetNames = [i.title for i in tmp]

# Because all sheets have the same metabolites, read in one sheet
sht = sheetNames[0]

writer = pd.ExcelWriter('ME1_mapped_metabolomics.xlsx', engine='xlsxwriter')
data = queryMetaboAnalyst(filename=name, sheet=sht)
modelMap = mapMetabolicModel(model)
mergedModelDataMap = matchModelAndData(data, modelMap)
PositionModel = mapMetabolitePositionsInModel(mergedModelDataMap, modelMap)
df = constructFinalDataset(PositionModel, filename, sht)

df = df['key_0'].rename(columns={'key_0':'Metabolites'})
df.to_excel(writer, sheet_name=sht, index=False)
writer.save()

Mapping metabolomics data to additional identifiers
Mapping metabolite names to PubChem database for synonym matching and ID retrieval.
                             source_id  ...              Name
sid                                     ...                  
53787130                       C058118  ...  2-deoxyadenosine
162088429                       102392  ...  2-deoxyadenosine
342500535                   WU050235CA  ...  2-deoxyadenosine
348274250                  HMDB0000101  ...  2-deoxyadenosine
383326151                 NSZB-A110030  ...  2-deoxyadenosine
404876780                   LQT-B18165  ...  2-deoxyadenosine
121474                           83258  ...  2-deoxyadenosine
427201                          141848  ...  2-deoxyadenosine
428054                          143510  ...  2-deoxyadenosine
250013057  OLXZPDWKRNYJJZ-UHFFFAOYSA-N  ...  2-deoxyadenosine

[10 rows x 5 columns]
          source_id  ...             Name
sid                  ...                 
117529240  AC